In [1]:


!pip install --quiet neo4j langchain-community langchain-experimental langchain-openai json-repair llama-index python-dotenv llama-index-graph-stores-neo4j

In [2]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://localhost:7691",
    username="neo4j",
    password="Sathviganu12$",
    refresh_schema=False
)

def clean_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    graph.query(query)

/var/folders/yh/7y6kspzd5zj6c33640swh6n40000gp/T/ipykernel_67898/1723572582.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [3]:

from langchain_core.documents import Document


text = """
Marie Curie, 7 November 1867 – 4 July 1934, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
Also, Robin Williams!
"""
documents = [Document(page_content=text)]
     

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers.llm import LLMGraphTransformer
from langchain.schema import Document
from langchain_openai import AzureChatOpenAI
from langchain.graphs import Neo4jGraph
import getpass
import os
import nest_asyncio
import asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Prompt for Azure OpenAI API key if not set in environment variables
if not os.getenv("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Azure OpenAI API key: ")

# Set the Azure OpenAI endpoint if not set in environment variables
if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    os.environ["AZURE_OPENAI_ENDPOINT"] = input("Azure OpenAI Endpoint: ")

# Initialize Azure OpenAI LLM
llm = AzureChatOpenAI(api_key=os.environ["AZURE_OPENAI_API_KEY"], model="gpt-4-0", temperature=0, api_version="2025-01-01-preview")

# Example text data
text = """Sarah is an employee at prismaticAI, a leading technology company based in Westside Valley. She has been working there for the past three years as a software engineer.
Michael is also an employee at prismaticAI, where he works as a data scientist. He joined the company two years ago after completing his graduate studies.
prismaticAI is a well-known technology company that specializes in developing cutting-edge software solutions and artificial intelligence applications. The company has a diverse workforce of talented individuals from various backgrounds.
Both Sarah and Michael are highly skilled professionals who contribute significantly to prismaticAI's success. They work closely with their respective teams to develop innovative products and services that meet the evolving needs of the company's clients."""

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_text(text)

# Convert text chunks to Document objects
documents = [Document(page_content=text) for text in texts]

# Initialize the transformer
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Person", "WORKS_AT", "Organization"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]
node_properties=["birth_date", "death_date"]
relationship_properties=["start_date"]

rels_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes, allowed_relationships=allowed_relationships, node_properties=node_properties, relationship_properties=relationship_properties, strict_mode=False)

# Initialize the Neo4j graph
graph = Neo4jGraph(
    url="bolt://localhost:7691",
    username="neo4j",
    password="Sathviganu12$",
    refresh_schema=False
)

# Define an asynchronous function to run the conversion and add to Neo4j
async def main():
    data = await rels_defined.aconvert_to_graph_documents(documents)
    
    # Print the graph documents
    for i, doc in enumerate(data):
        print(f"Graph Document {i+1}:\n{doc}\n")
    
    # Add the graph documents to the Neo4j graph
    graph.add_graph_documents(data)

    # Return the graph documents
    return data


# Run the asynchronous function
graph_documents = await main()

print("Final Graph Documents:")
for i, doc in enumerate(graph_documents):
    print(f"Graph Document {i+1}:\n{doc}\n")



/opt/anaconda3/envs/graphrag/lib/python3.9/site-packages/langchain_openai/chat_models/base.py:1413: UserWarning: Cannot use method='json_schema' with model gpt-4-0 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


Graph Document 1:
nodes=[Node(id='Sarah', type='Person', properties={}), Node(id='Prismaticai', type='Organization', properties={}), Node(id='Westside Valley', type='Location', properties={})] relationships=[Relationship(source=Node(id='Prismaticai', type='Organization', properties={}), target=Node(id='Westside Valley', type='Location', properties={}), type='IN_LOCATION', properties={}), Relationship(source=Node(id='Sarah', type='Person', properties={}), target=Node(id='Prismaticai', type='Organization', properties={}), type='WORKS_AT', properties={})] source=Document(metadata={}, page_content='Sarah is an employee at prismaticAI, a leading technology company based in Westside Valley. She has been working there for the past three years as a software engineer.')

Graph Document 2:
nodes=[Node(id='Michael', type='Person', properties={}), Node(id='Prismaticai', type='Organization', properties={})] relationships=[Relationship(source=Node(id='Michael', type='Person', properties={}), target=

In [5]:
!pip install py2neo

In [6]:
print(graph_documents)

[GraphDocument(nodes=[Node(id='Sarah', type='Person', properties={}), Node(id='Prismaticai', type='Organization', properties={}), Node(id='Westside Valley', type='Location', properties={})], relationships=[Relationship(source=Node(id='Prismaticai', type='Organization', properties={}), target=Node(id='Westside Valley', type='Location', properties={}), type='IN_LOCATION', properties={}), Relationship(source=Node(id='Sarah', type='Person', properties={}), target=Node(id='Prismaticai', type='Organization', properties={}), type='WORKS_AT', properties={})], source=Document(metadata={'id': 'b457d93643d911c68eb65ee6ecb9e0e5'}, page_content='Sarah is an employee at prismaticAI, a leading technology company based in Westside Valley. She has been working there for the past three years as a software engineer.')), GraphDocument(nodes=[Node(id='Michael', type='Person', properties={}), Node(id='Prismaticai', type='Organization', properties={})], relationships=[Relationship(source=Node(id='Michael', t

In [7]:
from py2neo import Graph, Node, Relationship

# Connect to Neo4j
py2neo_graph = Graph(
    "bolt://localhost:7691",
    auth=("neo4j", "Sathviganu12$")
)

# Assuming 'graph_documents' is a list of GraphDocument objects
for graph_doc in graph_documents:
    # Create nodes
    for node_data in graph_doc.nodes:
        node = Node(node_data.type, id=node_data.id, **node_data.properties)
        py2neo_graph.merge(node, node_data.type, "id")
    
    # Create relationships
    for rel_data in graph_doc.relationships:
        source_node = py2neo_graph.nodes.match(rel_data.source.type, id=rel_data.source.id).first()
        target_node = py2neo_graph.nodes.match(rel_data.target.type, id=rel_data.target.id).first()
        if source_node and target_node:
            relationship = Relationship(source_node, rel_data.type, target_node, **rel_data.properties)
            py2neo_graph.merge(relationship)

In [8]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore

In [9]:
from llama_index.core import StorageContext, KnowledgeGraphIndex
from llama_index.graph_stores import Neo4jGraphStore
from llama_index.schema import Document

# Add get_doc_id method to Document class if it does not exist
def get_doc_id(self):
    return self.metadata.get('id', None) or getattr(self, 'id_', None)

Document.get_doc_id = get_doc_id

# Add hash method to Document class if it does not exist
def hash(self):
    return hash(self.page_content)

Document.hash = hash

# Initialize the Neo4jGraphStore
graph_store = Neo4jGraphStore(
    username="neo4j",
    password="Sathviganu12$",
    url="bolt://localhost:7691",
    database="neo4j"
)

# Initialize the StorageContext
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

# Function to initialize the query engine
def initialize_query_engine(storage_context, llm):
    # Initialize the KnowledgeGraphRAGRetriever with the Azure OpenAI LLM
    graph_rag_retriever = KnowledgeGraphRAGRetriever(storage_context=storage_context, llm=llm, verbose=True)

    # Initialize the RetrieverQueryEngine with the Azure OpenAI LLM
    query_engine = RetrieverQueryEngine.from_args(retriever=graph_rag_retriever, llm=llm)

    return query_engine

# Example usage
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# Initialize Azure OpenAI LLM
llm = AzureChatOpenAI(api_key=azure_openai_api_key, model="gpt-4-0", temperature=0, api_version="2025-01-01-preview", endpoint=azure_openai_endpoint)

query_engine = initialize_query_engine(storage_context, llm)

# Example query
query = "Tell me about Sarah and Michael's work at prismaticAI."
response = query_engine.query(query)
print(response)

ImportError: cannot import name 'Neo4jGraphStore' from 'llama_index.graph_stores' (unknown location)